In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# 5.1 - Wprowadzenie do konwolucyjnych sieci neuronowych

Za chwilę zagłębimy się w zagadnienia teoretyczne zwiazane z konwolucyjnymi sieciami neuronowymi. Dlaczego sieci te tak dobrze sprawdzają się podczas przetwarzania obrazu? Zacznijmy od przyjrzenia się prostemu przykładowi sieci konwolucyjnej. Sieć ta służy do klasyfikacji zbioru cyfr MNIST. Problemem tym zajmowaliśmy się już wcześniej w rozdziale 2. — zastosowana wówczas sieć o gęstych połączeniach uzyskała dokładność testową na poziomie 97,8%. Mimo że sieć konwolucyjna, którą przedstawię za chwilę, będzie bardzo prosta, to uzyska ona o wiele lepszy wynik od modelu z rozdziału 2.

Poniższy kod przedstawia wygląd podstawowej sieci konwolucyjnej. Jest ona stosem warstw Conv2D i MaxPooling2D. Za chwilę opiszę szczegółowo działanie tych warstw.

Sieć neuronowa przyjmuje tensor o kształcie określanym przez wysokość obrazu, jego szerokość i kanały obrazu — nie wliczamy tutaj wymiaru próbki. W związku z tym sieć na przetwarzać obiekty wejściowe o rozmiarze (28, 28, 1) — taki jest właśnie format obrazów wchodzących w skład zbioru MNIST. W związku z tym do pierwszej warstwy sieci przekazujemy argument input_shape=(28, 28, 1).

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Wyświetlmy aktualną architekturę sieci:

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Na wyjściu każdej warstwy Conv2D i MaxPooling2D pojawia się trójwymiarowy tensor o kształcie (wysokość, szerokość, kanały). Wraz z zagłębianiem się asieci wysokość i szerokość mają tendencję do przyjmowania mniejszych wartości. Liczbę kanałów określa pierwszy argument przekazany do warstw Conv2D (możemy mieć np. 32 lub 64 kanały).

Kolejnym krokiem jest przekazanie ostatniego tensora wyjściowego o kształcie (3, 3, 64) do klasyfikatora w postaci sieci gęstej (ten typ sieci opisywałem wcześniej) — stosu warstw Dense. Klasyfikatory te przetwarzają jednowymiarowe wektory, a nasze dane mają postać trójwymiarowego tensora. W związku z tym musimy spłaszczyć nasze dane wyjściowe, a następnie dodać kilka górnych warstw Dense:


In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Klasyfikacja ma na celu dokonanie przyporządkowania do jednego z dziesięciu zbiorów, a więc będziemy korzystać z ostatniej warstwy generującej 10 wartości wyjściowych i użyjemy funkcji aktywacji softmax. Oto aktualna charakterystyka architektury sieci:

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Jak widać, tensory przed skierowaniem do dwóch warstw Dense, mając początkowo kształt (3, 3, 64) zostały spłaszczone do wektorów o kształcie (576, ).

Spróbujmy wytrenować sieć konwolucyjną na zbiorze MNIST. Możemy w tym celu ponownie użyć dużej części kodu przedstawionego wcześniej w rozdziale 2.



In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.1766 - acc: 0.9440     
Epoch 2/5
60000/60000 [==============================] - 7s - loss: 0.0462 - acc: 0.9855     
Epoch 3/5
60000/60000 [==============================] - 7s - loss: 0.0322 - acc: 0.9902     
Epoch 4/5
60000/60000 [==============================] - 7s - loss: 0.0241 - acc: 0.9926     
Epoch 5/5
60000/60000 [==============================] - 7s - loss: 0.0187 - acc: 0.9943     


Sprawdźmy działanie modelu na testowym zbiorze danych:

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [9]:
test_acc

0.99129999999999996

Gęsta sieć zastosowana w celu rozwiązania tego problemu w rozdziale 2. charakteryzowała się testową dokładnością na poziomie 97,8%, a podstawowa sieć konwolucyjna uzyskała wynik 99,3%. Względna wartość błędu została zmniejszona o 68%. Nieźle!